In [25]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import time
import os

tfds.disable_progress_bar()

In [26]:
# Mendapatkan path file untuk dataset Shakespeare dari URL
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

In [27]:
# membaca, lalu mendekode konten tersebut dari binary menjadi string menggunakan encoding ‘utf-8’
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# mencetak jumlah karakter dalam teks tersebut
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [28]:
# mencari jumlah karakter unik dalam teks
vocab = sorted(set(text)) # mengubah teks menjadi set untuk menghilangkan duplikat
print(f'{len(vocab)} unique characters') # mencetak jumlah karakter unik dalam teks

65 unique characters


In [29]:
example_texts = ['abcdefg', 'xyz'] # mendefinisikan list string yang akan diproses
chars = tf.strings.unicode_split (example_texts, input_encoding='UTF-8') # membagi setiap string dalam example_texts menjadi list karakter
chars # mencetak hasil dari operasi split

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [30]:
# membuat instance dari layer StringLookup dengan vocabulari yang ditentukan oleh vocab
ids_from_chars = tf.keras.layers.StringLookup(vocabulary=list(vocab), mask_token=None) 

ids = ids_from_chars(chars) # mengubah chars menjadi indeks integer menggunakan ids_from_chars
ids # mencetak array indeks integer yang mewakili chars

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [31]:
# membuat instance dari layer StringLookup dengan vocabulari yang ditentukan oleh ids_from_chars.get_vocabulary()
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

chars = chars_from_ids(ids) # mengubah ids kembali menjadi karakter menggunakan chars_from_ids
chars # mencetak array karakter yang mewakili ids

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [32]:
tf.strings.reduce_join(chars, axis=-1).numpy() # menggabungkan elemen-elemen dalam array chars menjadi string

array([b'abcdefg', b'xyz'], dtype=object)

In [33]:
# Membuat fungsi mengubah list indeks integer kembali menjadi string
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

# membagi teks menjadi list karakter dan hasilnya disimpan di variabel all_ids
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [34]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids) # membuat tf.data.Dataset dari all_ids

# Menampilkan karakter-karakter pertama dalam 10 tensor ID
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [35]:
# Menentukan panjang urutan yang diinginkan
seq_length = 100

# Membuat urutan dari dataset dengan panjang yang ditentukan
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

# Menampilkan urutan karakter pertama dalam satu batch
for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [36]:
# Menampilkan teks dari lima urutan pertama
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [37]:
# Mendefinisikan fungsi untuk membagi input dan target dari suatu urutan
def split_input_target(sequence):
  input_text = sequence[:-1]
  target_text = sequence[1:]
  return input_text, target_text

split_input_target(list("Tensorflow")) # Memanggil fungsi pada contoh urutan

dataset = sequences.map(split_input_target) # Membuat dataset dari urutan dengan input dan target yang terpisah

# Menampilkan contoh input dan target dari dataset
for input_example, target_example in dataset.take(1):
  print("Input :", text_from_ids(input_example).numpy())
  print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [38]:
# Ukuran batch
BATCH_SIZE = 64

# Ukuran buffer untuk mengacak dataset
BUFFER_SIZE = 10000

# Membuat dataset dengan mengacak, mengelompokkan, dan memuat dengan prefetch
dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset # Menampilkan dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [39]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Mendefinisikan kelas model yang merupakan turunan dari tf.keras.Model
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True,
                                   return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

# Membuat objek model menggunakan kelas yang telah didefinisikan
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [40]:
# Melakukan prediksi pada satu batch contoh input dari dataset
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

model.summary() # Menampilkan ringkasan arsitektur model

(64, 100, 66) # (batch_size, sequence_length, vocab_size)
Model: "my_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     multiple                  16896     
                                                                 
 gru_2 (GRU)                 multiple                  3938304   
                                                                 
 dense_2 (Dense)             multiple                  67650     
                                                                 
Total params: 4022850 (15.35 MB)
Trainable params: 4022850 (15.35 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [41]:
# Menghasilkan indeks teracak dari distribusi prediksi
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

sampled_indices

# Menampilkan contoh input dan prediksi karakter berikutnya
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"ke.\n\nCLARENCE:\nWe know thy charge, Brakenbury, and will obey.\n\nGLOUCESTER:\nWe are the queen's abject"

Next Char Predictions:
 b'be3nfkdjiU,zAEwzMbTbe:d&$fMerWKj?jFB [UNK]kYfK.jJ\ni$R!wRaQG-\nPFp!f,AcCOoh[UNK]afGixrPVCnQzFOZCwqI,e&Fo.nLcI[UNK]'


In [42]:
# Menggunakan SparseCategoricalCrossentropy sebagai fungsi loss
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# Menghitung rata-rata loss pada contoh batch
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1893554, shape=(), dtype=float32)


In [43]:
# Menghitung eksp dari rata-rata loss
tf.exp(example_batch_mean_loss).numpy()

65.98024

In [44]:
# mengkompilasi model dengan menentukan optimizer dan fungsi loss
model.compile(optimizer='adam', loss=loss)

In [45]:
# mendefinisikan direktori di mana checkpoint akan disimpan
checkpoint_dir = './training_checkpoints'

# mendefinisikan prefix untuk nama file checkpoint
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

# membuat callback ModelCheckpoint yang akan menyimpan bobot model ke file checkpoint setelah setiap epoch
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

EPOCHS = 20 # mendefinisikan jumlah epoch

# melatih model dengan dataset yang ditentukan selama jumlah epoch yang ditentukan, dan menyimpan bobot model setelah setiap epoch
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 [==============================] - 16s 66ms/step - loss: 2.7278
Epoch 2/20
172/172 [==============================] - 13s 64ms/step - loss: 1.9931
Epoch 3/20
172/172 [==============================] - 12s 61ms/step - loss: 1.7117
Epoch 4/20
172/172 [==============================] - 12s 62ms/step - loss: 1.5511
Epoch 5/20
172/172 [==============================] - 12s 61ms/step - loss: 1.4517
Epoch 6/20
172/172 [==============================] - 12s 61ms/step - loss: 1.3832
Epoch 7/20
172/172 [==============================] - 12s 61ms/step - loss: 1.3308
Epoch 8/20
172/172 [==============================] - 12s 62ms/step - loss: 1.2860
Epoch 9/20
172/172 [==============================] - 13s 64ms/step - loss: 1.2444
Epoch 10/20
172/172 [==============================] - 16s 62ms/step - loss: 1.2051
Epoch 11/20
172/172 [==============================] - 13s 62ms/step - loss: 1.1650
Epoch 12/20
172/172 [==============================] - 14s 62ms/step - loss: 1.1237
E

In [46]:
# mendefinisikan kelas CustomTraining yang mewarisi dari MyModel
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs): # mendefinisikan metode train_step yang akan dipanggil pada setiap langkah pelatihan
      inputs, labels = inputs # memisahkan data input dan label
      with tf.GradientTape() as tape: # membuat instance dari tf.GradientTape
          predictions = self(inputs, training=True) # melakukan forward pass melalui model (menggunakan metode call dari MyModel) untuk menghasilkan prediksi dari data input
          loss = self.loss(labels, predictions) # menghitung loss antara label dan prediksi menggunakan fungsi loss dari model
      grads = tape.gradient(loss, model.trainable_variables) # menghitung gradien loss terhadap semua variabel yang dapat dilatih dalam model
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables)) # menerapkan gradien ke variabel yang dapat dilatih menggunakan optimizer dari model

      return {'loss': loss} # mengembalikan dictionary yang berisi loss yang dihitung

In [47]:
# membuat instance dari kelas CustomTraining dengan parameter yang ditentukan   
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

# mengkompilasi model dengan optimizer Adam dan fungsi loss Sparse Categorical Crossentropy
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# melatih model dengan dataset yang ditentukan selama 1 epoch
model.fit(dataset, epochs=1)

172/172 [==============================] - 16s 62ms/step - loss: 2.7185


In [49]:
EPOCHS = 10 # mendefinisikan jumlah epoch

mean = tf.metrics.Mean() # menghitung rata-rata loss selama setiap epoch

# Melakukan pelatihan selama beberapa epoch
for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states() # Mereset nilai metrik Mean untuk setiap epoch
    for (batch_n, (inp, target)) in enumerate(dataset):# Iterasi melalui setiap batch dalam dataset
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])
        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # Menyimpan model setiap 5 epoch
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

# Menyimpan model setelah pelatihan selesai
model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.2056
Epoch 1 Batch 50 Loss 2.0146
Epoch 1 Batch 100 Loss 1.9134
Epoch 1 Batch 150 Loss 1.8436

Epoch 1 Loss: 1.9795
Time taken for 1 epoch 12.56 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7835
Epoch 2 Batch 50 Loss 1.7563
Epoch 2 Batch 100 Loss 1.6492
Epoch 2 Batch 150 Loss 1.6222

Epoch 2 Loss: 1.7049
Time taken for 1 epoch 12.54 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.6266
Epoch 3 Batch 50 Loss 1.5986
Epoch 3 Batch 100 Loss 1.5129
Epoch 3 Batch 150 Loss 1.4986

Epoch 3 Loss: 1.5456
Time taken for 1 epoch 12.17 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4749
Epoch 4 Batch 50 Loss 1.4303
Epoch 4 Batch 100 Loss 1.4335
Epoch 4 Batch 150 Loss 1.4588

Epoch 4 Loss: 1.4490
Time taken for 1 epoch 12.81 sec
_____________________________________________________________________